<h1>MassBalanceMachine Data Processing - Example for Retrieving Training Features for the Iceland Region (WGMS)</h1>
<p style='text-align: justify;'>
In this notebook, we will demonstrate the data processing workflow of the MassBalanceMachine using an example with glaciological data from Icelandic glaciers. This example will guide you through the data processing pipeline, which retrieves topographical and meteorological features for all points with glaciological surface mass balance observations, and transforms the data to a monthly resolution.
</p>
<p style='text-align: justify;'>
We begin by importing some basic libraries along with the <code>massbalancemachine</code> library. Next, we specify the location where we will store the files for the region of interest . The data used in this example is from the <a href='https://icelandicglaciers.is/index.html#/page/map'>Icelandic Glaciers inventory</a>, that was already preprocessed, to align with the WGMS data format we are using, in this <a href='https://github.com/ODINN-SciML/MassBalanceMachine/blob/main/notebooks/data_preprocessing.ipynb'>notebook</a>. You can also download data from the <a href='https://wgms.ch/data_databaseversions/'>WGMS database</a>, as you desire.
</p>

<b>Note:</b> WGMS data can contain errors or incorrect data, please check the data for correctness before using it.   

<b>Note:</b> All data, stake measurements and glaciers outlines, are expected to have the WGS84 CRS.

In [17]:
import pandas as pd
import geopandas as gpd
import xarray as xr
import massbalancemachine as mbm
example = False
thomas = True
WGMS = False
Swiss = False

<h2>1. Load your Target Surface Mass Balance Dataset and Retrieve RGI ID per Stake</h2>
<p style='text-align: justify;'>
In this step, we define and load our glaciological data from a region of interest. The WGMS dataset does not include RGI IDs by default, so we need to retrieve them from a glacier outline shapefile provided by the Randolph Glacier Inventory (v6). Each stake is then matched with an RGI ID. The RGI ID is necessary for the MassBalanceMachine to add additional topographical and meteorological features for training stage.
</p>
<p style='text-align: justify;'>
<b>How to Retrieve the Glacier Outlines:</b> Download the shapefiles for the region of interest from this <a href='https://daacdata.apps.nsidc.org/pub/DATASETS/nsidc0770_rgi_v6/'>link</a>. Extract the files and copy the .shp, .prj, .shx, and .dbf files in the correct directory so that you can use it with the Jupyter Notebook. Also, make sure you point to the correct directory and files in the next code cell.
</p>

<b>Note:</b> Data records that have an invalid FROM_DATE or TO_DATE, where the day is indicated as 99, are deleted from the dataset.

In [18]:
if example:
	# Specify the filename of the input file with the raw data
	target_data_fname = './example_data/iceland/files/iceland_wgms_dataset.csv'
	# Specify the shape filename of the glaciers outline obtained from RGIv6
	glacier_outline_fname = './example_data/iceland/glacier_outlines/06_rgi60_Iceland.shp'

	# Load the target data and the glacier outlines
	data = pd.read_csv(target_data_fname)
	glacier_outline = gpd.read_file(glacier_outline_fname)

<h3>1.1 Match the Stake Measurements with a RGI ID</h3>
<p style='text-align: justify;'>
Based on the location of the stake measurement given by POINT_LAT and POINT_LON, each data record is matched with the RGI ID for the glacier where the stake is located.
</p>

In [19]:
if example:
	# Get the RGI ID for each stake measurement for the region of interest
	data = mbm.data_processing.utils.get_rgi(data=data, glacier_outlines=glacier_outline)

<p style='text-align: justify;'>
Then, we can create a MassBalanceMachine `Dataset`, by using the loaded dataframe for Norway stake data together with the matched RGI IDs, as such: 
</p>

In [20]:
if example:
	# Provide the column name for the column that has the RGI IDs for each of the stakes
	dataset = mbm.Dataset(data=data, region_name='iceland', data_path='./example_data/iceland/files/')
elif thomas:
	data = pd.read_csv('./example_data/svalbard/Svalbard_seasonalMB.csv')
	dataset = mbm.Dataset(data=data, region_name='svalbard', data_path='./example_data/svalbard/')
elif WGMS:
	data = pd.read_csv('/mnt/c/IBDV/Data/MachineLearningGlaciology24/WGMS_mass_balance_point_RGI07.csv')
	# Fix data types
	data['RGIId'] = data.rgi
	data["YEAR"]=data["YEAR"].values.astype('int64')
	data["FROM_DATE"]=data["FROM_DATE"].values.astype('str')
	data["TO_DATE"]=data["TO_DATE"].values.astype('str')

	# Fix unknown dates
	data.loc[data['BALANCE_CODE']== 'BS',"FROM_DATE"]=data.loc[data['BALANCE_CODE']== 'BS',"FROM_DATE"].str.replace('9999$', '0501', regex=True)
	data.loc[data['BALANCE_CODE']== 'BW',"FROM_DATE"]=data.loc[data['BALANCE_CODE']== 'BW',"FROM_DATE"].str.replace('9999$', '1001', regex=True)
	data.loc[data['BALANCE_CODE']== 'BA',"FROM_DATE"]=data.loc[data['BALANCE_CODE']== 'BA',"FROM_DATE"].str.replace('9999$', '1001', regex=True)
	data.loc[data['BALANCE_CODE']== 'BS',"TO_DATE"]=data.loc[data['BALANCE_CODE']== 'BS',"TO_DATE"].str.replace('9999$', '0930', regex=True)
	data.loc[data['BALANCE_CODE']== 'BW',"TO_DATE"]=data.loc[data['BALANCE_CODE']== 'BW',"TO_DATE"].str.replace('9999$', '0430', regex=True)
	data.loc[data['BALANCE_CODE']== 'BA',"TO_DATE"]=data.loc[data['BALANCE_CODE']== 'BA',"TO_DATE"].str.replace('9999$', '0930', regex=True)

	# Remove empty coordinates
	data = data.loc[~data['POINT_LAT'].isna()]
	data.reset_index(inplace=True)
elif Swiss:
	data = pd.read_csv("/mnt/c/IBDV/Data/MassBalance/GLAMOS_Switzerland/GLAMOS_Switzerland_WinterBalance.csv")
	dataset = mbm.Dataset(data=data, region_name='swiss', data_path='./example_data/switzerland/')

	

In [21]:
if WGMS:
	# Remove BA if BS and BW are available
	data["id"] = data.groupby(data[["YEAR", "POINT_LAT"]].apply(frozenset, axis=1)).ngroup() + 1
	for i in data['id'].unique():
		codes = data.loc[data['id'] == i,'BALANCE_CODE']
		if len(codes) == 3:
			data = data.drop(codes.index[codes == 'BA'])

		if (len(codes) == 2) and ('BA' in codes.values):
			if 'BS' in codes.values:
				data.loc[codes.index[codes == 'BA'],"POINT_BALANCE"] = data.loc[codes.index[codes == 'BA'],"POINT_BALANCE"].values - data.loc[codes.index[codes == 'BS'],"POINT_BALANCE"].values
				data.loc[codes.index[codes == 'BA'],"TO_DATE"] = data.loc[codes.index[codes == 'BA'],"YEAR"].astype('str').values+'0430'
				data.loc[codes.index[codes == 'BA'],"BALANCE_CODE"] = 'BW'
			if 'BW' in codes.values:
				data.loc[codes.index[codes == 'BA'],"POINT_BALANCE"] = data.loc[codes.index[codes == 'BA'],"POINT_BALANCE"].values - data.loc[codes.index[codes == 'BW'],"POINT_BALANCE"].values
				data.loc[codes.index[codes == 'BA'],"FROM_DATE"] = data.loc[codes.index[codes == 'BA'],"YEAR"].astype('str').values+'0501'
				data.loc[codes.index[codes == 'BA'],"BALANCE_CODE"] = 'BS'
	data.reset_index(inplace=True)

	# Provide the column name for the column that has the RGI IDs for each of the stakes
	dataset = mbm.Dataset(data=data, region_name='svalbard', data_path='./example_data/svalbard/')


In [22]:
dataset.data

,index,NAME,YEAR,POINT_ID,POINT_LAT,POINT_LON,POINT_ELEVATION,FROM_DATE,TO_DATE,POINT_BALANCE,BALANCE_CODE,RGIId
0,0,BRG,1967,NaN,78.904665,11.827670,141,19670501,19670930,-1610.0,BS,RGI60-07.00504
1,0,BRG,1967,NaN,78.901834,11.828051,155,19670501,19670930,-1550.0,BS,RGI60-07.00504
2,0,BRG,1967,NaN,78.894377,11.843428,201,19670501,19670930,-1420.0,BS,RGI60-07.00504
3,0,BRG,1967,NaN,78.892639,11.858351,224,19670501,19670930,-1270.0,BS,RGI60-07.00504
4,0,BRG,1967,NaN,78.890686,11.875982,264,19670501,19670930,-1290.0,BS,RGI60-07.00504
...,...,...,...,...,...,...,...,...,...,...,...,...
3716,0,NSB,2018,NaN,78.675723,17.164311,463,20171001,20180430,40.0,BW,RGI60-07.00570
3717,0,NSB,2018,NaN,78.692834,17.153921,531,20171001,20180430,220.0,BW,RGI60-07.00570
3718,0,NSB,2018,NaN,78.720934,17.251063,584,20171001,20180430,460.0,BW,RGI60-07.00570
3719,0,NSB,2018,NaN,78.790345,17.450070,1050,20171001,20180430,880.0,BW,RGI60-07.00570


<h2>2. Get Topographical Features per Stake Measurement</h2>
<p style='text-align: justify;'>
Once we have created a <code>Dataset</code>, the first thing we can do is to add topographical data in our dataset. This can be done automatically with the MassBalanceMachine (which calls OGGM) by doing the following:
</p>

In [23]:
# Specify the topographical features of interest 
# Please see the OGGM documentation what variables are available: https://oggm.org/tutorials/stable/notebooks/10minutes/machine_learning.html ('topo', 'slope_factor', 'dis_from_border')
voi_topographical = ['aspect', 'slope','dis_from_border']

# Retrieve the topographical features for each stake measurement and add them to the dataset
#dataset.get_topo_features(vois=voi_topographical)

dataset.data = pd.read_csv('./example_data/svalbard/svalbard_topographical_features.csv')

display(dataset.data)


,index,NAME,YEAR,POINT_ID,POINT_LAT,POINT_LON,POINT_ELEVATION,FROM_DATE,TO_DATE,POINT_BALANCE,BALANCE_CODE,RGIId,aspect,slope,dis_from_border,accumulation,curvature,vicinity
0,0,BRG,1967,NaN,78.904665,11.827670,141,19670501,19670930,-1610.0,BS,RGI60-07.00504,6.279406,0.107976,120.74767,0.030651,-0.000108,800.307422
1,0,BRG,1967,NaN,78.901834,11.828051,155,19670501,19670930,-1550.0,BS,RGI60-07.00504,6.114081,0.040374,378.00000,0.016187,-0.000305,844.456668
2,0,BRG,1967,NaN,78.894377,11.843428,201,19670501,19670930,-1420.0,BS,RGI60-07.00504,5.628025,0.041985,345.76870,0.064932,-0.000277,1283.565308
3,0,BRG,1967,NaN,78.892639,11.858351,224,19670501,19670930,-1270.0,BS,RGI60-07.00504,5.076932,0.095281,341.52600,0.097249,0.000049,1145.512939
4,0,BRG,1967,NaN,78.890686,11.875982,264,19670501,19670930,-1290.0,BS,RGI60-07.00504,5.054092,0.107405,381.83768,0.215570,-0.000434,924.331055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3716,0,NSB,2018,NaN,78.675723,17.164311,463,20171001,20180430,40.0,BW,RGI60-07.00570,2.752771,0.024311,721.11020,0.229626,0.003698,2203.571631
3717,0,NSB,2018,NaN,78.692834,17.153921,531,20171001,20180430,220.0,BW,RGI60-07.00570,2.436726,0.027955,1697.05630,0.036651,-0.001989,2430.195312
3718,0,NSB,2018,NaN,78.720934,17.251063,584,20171001,20180430,460.0,BW,RGI60-07.00570,4.309643,0.019715,848.52814,0.094963,0.000782,3716.912158
3719,0,NSB,2018,NaN,78.790345,17.450070,1050,20171001,20180430,880.0,BW,RGI60-07.00570,3.244827,0.049891,3224.90300,0.366606,-0.000546,11654.339160


In [8]:
#from pandas.tseries.offsets import DateOffset
#dataset.data['TO_DATE'] = pd.to_datetime(dataset.data['TO_DATE'], format = '%Y-%m-%d')
#dataset.data['FROM_DATE'] = pd.to_datetime(, format = '%Y-%m-%d') #dataset.data['TO_DATE'] - DateOffset(months=6)

#dataset.data

<h2>3. Get Meteorological Features per Stake Measurement</h2>
<p style='text-align: justify;'>
Once we have the topographical data, we can add the necessary climate data for the dataset. This is done by pulling that from ERA5-Land database for the region of interest. Before the climate data is matched with the stake measurements, we need to manually download the climate data for the region of interest from <a href='https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land-monthly-means?tab=form'>Climate Copernicus website</a>. Check the following options:
</p>


<table border="1" style="margin-top: 20px; margin-bottom: 20px; border-collapse: collapse; width: 100%;">
    <tr>
        <th style="padding: 10px; text-align: left;">Field</th>
        <th style="padding: 10px; text-align: left;">Details</th>
    </tr>
    <tr>
        <td style="padding: 10px;"><strong>Product type:</strong></td>
        <td style="padding: 10px;"><em>Monthly averaged reanalysis</em></td>
    </tr>
    <tr>
        <td style="padding: 10px;"><strong>Variable:</strong></td>
        <td style="padding: 10px;">
            <ul style="margin: 0; padding-left: 20px;">
                <li><em>2m temperature</em> (t2m)</li>
                <li><em>Total precipitation</em> (tp)</li>
                <li><em>Surface latent heat flux</em> (slhf)</li>
                <li><em>Surface sensible heat flux</em> (sshf)</li>
                <li><em>Surface solar radiation downwards</em> (ssrd)</li>
                <li><em>Forecast albedo</em> (fal)</li>
                <li><em>Surface net thermal radiation</em> (str)</li>
                <li>...</li>
                <li>You can explore additional options as you prefer. In this example, we use the variables listed above.</li>
            </ul>
        </td>
    </tr>
    <tr>
        <td style="padding: 10px;"><strong>Year:</strong></td>
        <td style="padding: 10px;"><em>Select all</em></td>
    </tr>
    <tr>
        <td style="padding: 10px;"><strong>Month:</strong></td>
        <td style="padding: 10px;"><em>Select all</em></td>
    </tr>
    <tr>
        <td style="padding: 10px;"><strong>Time:</strong></td>
        <td style="padding: 10px;"><em>Select all</em></td>
    </tr>
    <tr>
        <td style="padding: 10px;"><strong>Geographical area:</strong></td>
        <td style="padding: 10px;">
            <em>Either download for the entire Earth, or specify the coordinates for a bounding box in the region of interest. For the region of interest, provide a North, East, South, and West coordinate, specifying the two corners of the bounding box.</em>
        </td>
    </tr>
    <tr>
        <td style="padding: 10px;"><strong>Format:</strong></td>
        <td style="padding: 10px;"><em>NetCDF-3</em></td>
    </tr>
</table>


<p style='text-align: justify;'>
Then click <i>Submit Request</i> (after you have registered or logged into your account). Please be aware that downloading this data can take up to several hours, depending on the number of variables, timescales, and the area selected. Once the download is complete, place the netCDF file in the correct directory and rename it accordingly.
</p>
<p style='text-align: justify;'>
Additionally, we need the _geopotential height_ as an extra variable in our dataset. We will calculate a new variable by determining the difference between the geopotential height and the elevation at the stake measurement. The purpose of this height difference is to encourage the model to use it for downscaling the ERA5Land climate data, rather than relying on the lapse rate. The geopotential is downloaded from <a href='https://confluence.ecmwf.int/display/CKB/ERA5-Land%3A+data+documentation#ERA5Land:datadocumentation-parameterlistingParameterlistings'>here</a>, and is already included in this example.
</p>   

In [24]:
# Specify the files of the climate data, that will be matched with the coordinates of the stake data
if example:
	climate_data = './example_data/iceland/climate/era5_monthly_averaged_data.nc'
	geopotential_data = './example_data/iceland/climate/era5_geopotential_pressure.nc'
else:
	#climate_data = './example_data/svalbard/era5_monthly_Svalbard.nc'
	#climate_data = './example_data/svalbard/era5_svalbard_incwindUV.nc'
	#geopotential_data = './example_data/iceland/climate/era5_geopotential_pressure.nc'
	geopotential_data = './example_data/svalbard/carra_east_geopotential_pressure.nc'
	climate_data = './example_data/svalbard/carra_monthly_east_rotatedgrid.nc'


#dataset.data.drop(dataset.data.tail(3000).index, inplace = True)

# Match the climate features, from the ERA5Land netCDF file, for each of the stake measurement dataset
dataset.get_climate_features(climate_data=climate_data, geopotential_data=geopotential_data, era5=False)
display(dataset.data)

,index,NAME,YEAR,POINT_ID,POINT_LAT,POINT_LON,POINT_ELEVATION,FROM_DATE,TO_DATE,POINT_BALANCE,BALANCE_CODE,RGIId,aspect,slope,dis_from_border,accumulation,curvature,vicinity,ssrd_oct,ssrd_nov,ssrd_dec,ssrd_jan,ssrd_feb,ssrd_mar,ssrd_apr,ssrd_may,ssrd_jun,ssrd_jul,ssrd_aug,ssrd_sep,str_oct,str_nov,str_dec,str_jan,str_feb,str_mar,str_apr,str_may,str_jun,str_jul,str_aug,str_sep,tp_oct,tp_nov,tp_dec,tp_jan,tp_feb,tp_mar,tp_apr,tp_may,tp_jun,tp_jul,tp_aug,tp_sep,al_oct,al_nov,al_dec,al_jan,al_feb,al_mar,al_apr,al_may,al_jun,al_jul,al_aug,al_sep,sshf_oct,sshf_nov,sshf_dec,sshf_jan,sshf_feb,sshf_mar,sshf_apr,sshf_may,sshf_jun,sshf_jul,sshf_aug,sshf_sep,slhf_oct,slhf_nov,slhf_dec,slhf_jan,slhf_feb,slhf_mar,slhf_apr,slhf_may,slhf_jun,slhf_jul,slhf_aug,slhf_sep,t2m_oct,t2m_nov,t2m_dec,t2m_jan,t2m_feb,t2m_mar,t2m_apr,t2m_may,t2m_jun,t2m_jul,t2m_aug,t2m_sep,ALTITUDE_CLIMATE,ELEVATION_DIFFERENCE
0,0,BRG,1967,NaN,78.904665,11.827670,141,19670501,19670930,-1610.0,BS,RGI60-07.00504,6.279406,0.107976,120.74767,0.030651,-0.000108,800.307422,202304.171875,2.023042e+05,2.023042e+05,2.023042e+05,2.023042e+05,202304.171875,202304.171875,202304.171875,202304.171875,202304.171875,202304.171875,202304.171875,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,334.169434,193.169434
1,0,BRG,1967,NaN,78.901834,11.828051,155,19670501,19670930,-1550.0,BS,RGI60-07.00504,6.114081,0.040374,378.00000,0.016187,-0.000305,844.456668,202304.171875,2.023042e+05,2.023042e+05,2.023042e+05,2.023042e+05,202304.171875,202304.171875,202304.171875,202304.171875,202304.171875,202304.171875,202304.171875,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,334.169434,179.169434
2,0,BRG,1967,NaN,78.894377,11.843428,201,19670501,19670930,-1420.0,BS,RGI60-07.00504,5.628025,0.041985,345.76870,0.064932,-0.000277,1283.565308,214176.828125,2.141768e+05,2.141768e+05,2.141768e+05,2.141768e+05,214176.828125,214176.828125,214176.828125,214176.828125,214176.828125,214176.828125,214176.828125,-178250.828125,-178250.828125,-178250.828125,-178250.828125,-178250.828125,-178250.828125,-178250.828125,-178250.828125,-178250.828125,-178250.828125,-178250.828125,-178250.828125,0.047541,0.047541,0.047541,0.047541,0.047541,0.047541,0.047541,0.047541,0.047541,0.047541,0.047541,0.047541,49.864708,49.864708,49.864708,49.864708,49.864708,49.864708,49.864708,49.864708,49.864708,49.86470

In [12]:
pd.set_option("display.max_columns", None)
display(dataset.data)
dataset.data.columns

,index,NAME,YEAR,POINT_ID,POINT_LAT,POINT_LON,POINT_ELEVATION,FROM_DATE,TO_DATE,POINT_BALANCE,BALANCE_CODE,RGIId,aspect,slope,dis_from_border,accumulation,curvature,vicinity,ssrd_oct,ssrd_nov,ssrd_dec,ssrd_jan,ssrd_feb,ssrd_mar,ssrd_apr,ssrd_may,ssrd_jun,ssrd_jul,ssrd_aug,ssrd_sep,str_oct,str_nov,str_dec,str_jan,str_feb,str_mar,str_apr,str_may,str_jun,str_jul,str_aug,str_sep,tp_oct,tp_nov,tp_dec,tp_jan,tp_feb,tp_mar,tp_apr,tp_may,tp_jun,tp_jul,tp_aug,tp_sep,al_oct,al_nov,al_dec,al_jan,al_feb,al_mar,al_apr,al_may,al_jun,al_jul,al_aug,al_sep,sshf_oct,sshf_nov,sshf_dec,sshf_jan,sshf_feb,sshf_mar,sshf_apr,sshf_may,sshf_jun,sshf_jul,sshf_aug,sshf_sep,slhf_oct,slhf_nov,slhf_dec,slhf_jan,slhf_feb,slhf_mar,slhf_apr,slhf_may,slhf_jun,slhf_jul,slhf_aug,slhf_sep,t2m_oct,t2m_nov,t2m_dec,t2m_jan,t2m_feb,t2m_mar,t2m_apr,t2m_may,t2m_jun,t2m_jul,t2m_aug,t2m_sep,ALTITUDE_CLIMATE,ELEVATION_DIFFERENCE,MONTHS,N_MONTHS,ID
0,0,BRG,1967,NaN,78.904665,11.827670,141,1967-05-01,1967-09-30,-1610.0,BS,RGI60-07.00504,6.279406,0.107976,120.74767,0.030651,-0.000108,800.307422,202304.171875,2.023042e+05,2.023042e+05,2.023042e+05,2.023042e+05,202304.171875,202304.171875,202304.171875,202304.171875,202304.171875,202304.171875,202304.171875,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,334.169434,193.169434,"[may, jun, jul, aug, sep]",4,0
1,0,BRG,1967,NaN,78.901834,11.828051,155,1967-05-01,1967-09-30,-1550.0,BS,RGI60-07.00504,6.114081,0.040374,378.00000,0.016187,-0.000305,844.456668,202304.171875,2.023042e+05,2.023042e+05,2.023042e+05,2.023042e+05,202304.171875,202304.171875,202304.171875,202304.171875,202304.171875,202304.171875,202304.171875,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,-184984.140625,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,0.047188,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,38.075321,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,68248.773438,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,-152152.921875,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,274.327728,334.169434,179.169434,"[may, jun, jul, aug, sep]",4,1
2,0,BRG,1967,NaN,78.894377,11.843428,201,1967-05-01,1967-09-30,-1420.0,BS,RGI60-07.00504,5.628025,0.041985,345.76870,0.064932,-0.000277,1283.565308,214176.828125,2.141768e+05,2.141768e+05,2.141768e+05,2.141768e+05,214176.828125,214176.828125,214176.828125,214176.828125,214176.828125,214176.828125,214176.828125,-178250.828125,-178250.828125,-178250.828125,-178250.828125,-178250.828125,-178250.828125,-178250.828125,-178250.828125,-178250.828125,-178250.828125,-178250.828125,-178250.828125,0.047541,0.047541,0.047541,0.047541,0.047541,0.047541,0.047541,0.047541,0.047541,0.047541,0.047541,0.047541,49.

Index(['index', 'NAME', 'YEAR', 'POINT_ID', 'POINT_LAT', 'POINT_LON',
       'POINT_ELEVATION', 'FROM_DATE', 'TO_DATE', 'POINT_BALANCE',
       ...
       't2m_may', 't2m_jun', 't2m_jul', 't2m_aug', 't2m_sep',
       'ALTITUDE_CLIMATE', 'ELEVATION_DIFFERENCE', 'MONTHS', 'N_MONTHS', 'ID'],
      dtype='object', length=107)

<h2>4. Transform Data to Monthly Time Resolution</h2>
<p style='text-align: justify;'>
Finally, we need to transform the dataset into a monthly resolution, accounting for a variable period for the SMB target data. This will be done in order to perform SMB predictions at a monthly time step, which then will be integrated both in time and space to match the available glaciological and geodetic SMB observations for the training. Please specify the climate variables used in the previous step in the list below. Consult the <a href='https://confluence.ecmwf.int/display/CKB/ERA5-Land%3A+data+documentation'>documentation</a> of all short names of the climate variables.
</p>

In [25]:
# Specify the short names of the climate variables available in the dataset
vois_climate = ['t2m', 'tp', 'slhf', 'sshf', 'ssrd', 'al', 'str'] #,'u10','v10'

# For each record, convert to a monthly time resolution
dataset.convert_to_monthly(vois_climate=vois_climate, vois_topographical=voi_topographical+['accumulation','curvature','vicinity'])

Finally, we can take a look at the dataset which will be used for training.

In [15]:
display(dataset.data)

,YEAR,POINT_LON,POINT_LAT,POINT_BALANCE,ALTITUDE_CLIMATE,ELEVATION_DIFFERENCE,RGIId,POINT_ID,ID,N_MONTHS,MONTHS,aspect,slope,dis_from_border,accumulation,curvature,vicinity,t2m,tp,slhf,sshf,ssrd,al,str
0,1967,11.827670,78.904665,-1610.0,334.169434,193.169434,RGI60-07.00504,NaN,0,4,may,6.279406,0.107976,120.74767,0.030651,-0.000108,800.307422,274.327728,0.047188,-152152.921875,68248.773438,2.023042e+05,38.075321,-184984.140625
1,1967,11.827670,78.904665,-1610.0,334.169434,193.169434,RGI60-07.00504,NaN,0,4,jun,6.279406,0.107976,120.74767,0.030651,-0.000108,800.307422,274.327728,0.047188,-152152.921875,68248.773438,2.023042e+05,38.075321,-184984.140625
2,1967,11.827670,78.904665,-1610.0,334.169434,193.169434,RGI60-07.00504,NaN,0,4,jul,6.279406,0.107976,120.74767,0.030651,-0.000108,800.307422,274.327728,0.047188,-152152.921875,68248.773438,2.023042e+05,38.075321,-184984.140625
3,1967,11.827670,78.904665,-1610.0,334.169434,193.169434,RGI60-07.00504,NaN,0,4,aug,6.279406,0.107976,120.74767,0.030651,-0.000108,800.307422,274.327728,0.047188,-152152.921875,68248.773438,2.023042e+05,38.075321,-184984.140625
4,1967,11.827670,78.904665,-1610.0,334.169434,193.169434,RGI60-07.00504,NaN,0,4,sep,6.279406,0.107976,120.74767,0.030651,-0.000108,800.307422,274.327728,0.047188,-152152.921875,68248.773438,2.023042e+05,38.075321,-184984.140625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22428,2018,17.453729,78.811326,550.0,1003.143494,-129.856506,RGI60-07.00570,NaN,3720,6,dec,2.714824,0.050351,1019.80390,0.147174,-0.000530,NaN,257.170319,0.091374,-2484.843750,51883.800781,2.823156e-10,NaN,-132569.609375
22429,2018,17.453729,78.811326,550.0,1003.143494,-129.856506,RGI60-07.00570,NaN,3720,6,jan,2.714824,0.050351,1019.80390,0.147174,-0.000530,NaN,266.487274,0.314881,-5762.536621,-3904.708252,1.881845e-10,NaN,-22980.056641
22430,2018,17.453729,78.811326,550.0,1003.143494,-129.856506,RGI60-07.00570,NaN,3720,6,feb,2.714824,0.050351,1019.80390,0.147174,-0.000530,NaN,257.106293,0.003901,9360.854492,172236.656250,1.881788e-10,NaN,-192449.078125
22431,2018,17.453729,78.811326,550.0,1003.143494,-129.856506,RGI60-07.00570,NaN,3720,6,mar,2.714824,0.050351,1019.80390,0.147174,-0.000530,NaN,256.580048,0.057099,20826.458984,134608.296875,9.976534e+04,86.022881,-99670.687500


<p style='text-align: justify;'>
We have finished preprocessing the training data for our machine learning model. You can either explore the dataset in this <a href='https://github.com/ODINN-SciML/MassBalanceMachine/blob/main/notebooks/date_exploration.ipynb'>notebook</a> or continue with model training in this <a href='https://github.com/ODINN-SciML/MassBalanceMachine/blob/main/notebooks/xgboost_model_training.ipynb'>notebook</a>.
</p>